In [10]:
from itertools import starmap
from functools import partial
from collections import namedtuple
import random
import traitlets
import datetime

from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox,\
                       Textarea, Image, Label, HTML, Dropdown,\
                       IntProgress, Tab, IntText, Select
import pandas
import jinja2
from pony import orm

In [ ]:
import db
db.use_db('course.sqlite')

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 2,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

# Widget definitions and interaction

In [126]:
class AppState(traitlets.HasTraits):
    students = traitlets.List(traitlets.Unicode(), read_only=True)
    graders = traitlets.List(traitlets.Unicode(), read_only=True)
    
    exam_id = traitlets.Integer()

    submission_id = traitlets.Integer()
    student = traitlets.Unicode()

    # Default grader_id = 0 is definitely not in the database.
    # Here we are abusing 1-based indexing and the internal db structure.
    grader_id = traitlets.Integer()

    problem_id = traitlets.Integer()
    n_solutions = traitlets.Integer()
    n_graded = traitlets.Integer()
    feedback_options = traitlets.List(trait=traitlets.Unicode())

    selected_feedback = traitlets.List(trait=traitlets.Unicode())
    remarks = traitlets.Unicode()
    # --- Defaults and validation ---
    ## students
    @traitlets.default('students')
    def _default_students(self):
        with orm.db_session:
            return tuple(starmap('{} ({} {})'.format, 
                                 orm.select((s.id, s.first_name, s.last_name)
                                            for s in db.Student)))

    ## graders
    @traitlets.default('graders')
    def _default_graders(self):
        with orm.db_session:
            graders = list(starmap('{} {}'.format, 
                                   orm.select((g.first_name, g.last_name)
                                              for g in db.Grader).order_by(lambda g: g.id)))
        return ["None"] + graders

    ## exam_id
    @traitlets.default('exam_id')
    def _default_exam_id(self):
        with orm.db_session:
            return db.Exam.select().first().id
    
    @traitlets.validate('exam_id')
    def _valid_exam(self, proposal):
        with orm.db_session():
            if db.Exam.get(id=proposal['value']) is None:
                raise traitlets.TraitError('Unknown exam id.')
        return proposal['value']
    
    ## submission_id
    @traitlets.default('submission_id')
    def _default_submission_id(self):
        with orm.db_session:
            return orm.select(s.id for s in db.Submission 
                              if s.exam.id == self.exam_id).order_by(lambda x:x).first()
    
    @traitlets.validate('submission_id')
    def _valid_submission_id(self, proposal):
        with orm.db_session():
            sub = db.Submission.get(id=proposal['value'])
            if sub is None:
                raise traitlets.TraitError('Unknown submission id.')
            if sub.exam.id != self.exam_id:
                raise traitlets.TraitError('Submission from a different exam.')
        return proposal['value']
    
    ## problem_id
    @traitlets.default('problem_id')
    def _default_problem_id(self):
        with orm.db_session:
            return orm.select(p.id for p in db.Problem 
                              if p.exam == db.Exam[self.exam_id]).order_by(lambda x:x).first()
    
    @traitlets.validate('problem_id')
    def _valid_problem_id(self, proposal):
        with orm.db_session():
            prob = db.Problem.get(id=proposal['value'])
            if prob is None:
                raise traitlets.TraitError('Unknown problem id.')
            if prob.exam.id != self.exam_id:
                raise traitlets.TraitError('Problem from a different exam.')
        return proposal['value']
    
    ## n_solutions
    @traitlets.default('n_solutions')
    def _default_n_solutions(self):
        with orm.db_session:
            return orm.count(s for s in db.Solution if s.problem.id == self.problem_id)

    ## n_graded
    @traitlets.default('n_solutions')
    def _default_n_graded(self):
        with orm.db_session:
            return orm.count(s for s in db.Solution if s.problem.id == self.problem_id 
                                                       and s.graded_at is not None)
    
    ## grader_id
    @traitlets.validate('grader_id')
    def _valid_grader_id(self, proposal):
        new = proposal['value']
        if new == -1:
            return new
        else:
            with orm.db_session:
                if db.Grader.get(id=new) is None:
                    raise traitlets.TraitError('Non-existent grader id.')
        return new
    
    ## feedback_options
    def _default_feedback_options(self):
        with orm.db_session:
            return list(orm.select(f.text for f in db.FeedbackOption
                                   if f.problem.id == self.problem_id).order_by('f.id'))
    
    ## selected_feedback
    def _default_selected_feedback(self):
        with orm.db_session:
            sol = db.Solution.get(submission=self.submission_id, problem=self.problem_id)
            if sol is None:
                return []
            return [f.text for f in sol.feedback]
    
    ## remarks
    @traitlets.default('remarks')
    def _default_remarks(self):
        with orm.db_session:
            sol = db.Solution.get(submission=self.submission_id, problem=self.problem_id)
            if sol is None:
                return ''
            return sol.remarks or ''

    ## student
    @traitlets.default('student')
    def _default_student(self):
        with orm.db_session:
            student = db.Submission[self.submission_id].student
            if student is None:
                return "MISSING"
            else:
                return "{} ({}, {})".format(student.id, student.first_name, student.last_name)
    
    # --- Relations between traits ---
    @traitlets.observe('exam_id')
    def _change_exam(self, change):
        self.submission_id = self._default_submission_id()
        self.problem_id = self._default_problem_id()
    
    @traitlets.observe('problem_id', 'submission_id')
    def _change_solution(self, change):
        self.commit_grading(**{change['name']: change['old']})
        self.student = self._default_student()
    
    @traitlets.observe('problem_id')
    def _change_problem(self, change):
        self.feedback_options = self._default_feedback_options()
        self.selected_feedback = self._default_selected_feedback()
    
    # --- Writing into database ---
    def commit_grading(self, submission_id=None, problem_id=None):
        """Commit grading to the database"""
        if submission_id is None:
            submission_id = self.submission_id
        if problem_id is None:
            problem_id = self.problem_id

        # Should do nothing when the grader is missing.
        if not self.grader:
            return
        
        with orm.db_session:
            solution = db.Solution.get(submission=submission_id, problem=problem_id)
            feedback = solution.feedback
            remarks = solution.remarks
            
        # TODO: finish this

# # save the selected feedback
# def save_feedback():
#     if grader_id is None:
#         return  # should not save if the grader is not selected
#     selected_feedback = set(options_feedback.value)
#     if selected_feedback and selected_feedback == loaded_feedback and remarks.value == loaded_remarks:
#         return  # nothing was changed and options are non-empty -- do not save
#     with orm.db_session:
#         solution = db.Solution.get(submission=submission_id, problem=problem_id)
#         solution.feedback =  list(orm.select(fb for fb in db.FeedbackOption if fb.text in selected_feedback
#                                              and fb.problem.id == problem_id))
#         solution.remarks = remarks.value
#         if len(solution.feedback):
#             solution.graded_by = db.Grader[grader_id]
#             solution.graded_at = datetime.datetime.now()
#         else:
#             solution.graded_by = None
#             solution.graded_at = None

    
    def commit_feedback(self):
        pass
    
    # --- Navigation
    def next_submission(self):
        with orm.db_session:
            subs = orm.select(s.id for s in db.Submission
                              if s.exam.id == self.exam_id
                                 and s.id > self.submission.id)
            result = subs.order_by(lambda x: x).first()
            if result is None:
                self.submission_id = self._default_submission_id()
            else:
                self.submission_id = result
    
    def previous_submission(self):        
        with orm.db_session:
            subs = orm.select(s.id for s in db.Submission
                              if s.exam.id == self.exam_id
                                 and s.id < self.submission.id)

            result = subs.order_by(lambda x: x).last()
            if result is None:
                subs = orm.select(s.id for s in db.Submission if s.exam.id == self.exam_id)
                result = subs.order_by(lambda x: x).last()
            
            self.submission_id = result

    def next_ungraded(self):
        with orm.db_session:
            subs = orm.select(s.id for s in db.Submission
                              if s.exam.id == self.exam_id
                                 and s.id > self.submission.id
                                 and db.Solution.get(submission=s.id, 
                                                     problem=self.problem_id).graded_at is not None)
            result = subs.order_by(lambda x: x).first()
            if result is None:
                return

            self.submission_id = result

    def jump_to_student(self, nr):
        with orm.db_session:
            sub_id = db.Submission.get(exam=self.exam_id, student=nr).id
        if sub_id is None:
            return
        
        self.submission_id = sub_id

In [127]:
state = AppState()

In [87]:
traitlets.observe?

In [86]:
state._default_problem_id()

In [ ]:
state = AppState()

## Solution selection (submission x problem) matrix
prev_stud = Button(icon='fa-step-backward', description=' previous')
next_stud = Button(icon='fa-step-forward', description=' next')
next_ungraded_stud = Button(icon='fa-forward', description=' next ungraded')
jump_to_stud = Text(description='Jump to', placeholder='student # or name')


jump_to_problem = Dropdown(description='Jump to', placeholder='problem #', options=problems)

## Solution and FeedbackOption
options_feedback = SelectMultiple(name='feedback')
remarks = Textarea(placeholder='Student-specific feedback')
answer = Image(format='png', width='100%')

## info
graded_label = HTML()
graded_label.template = '<h5>Graded by: {} @ {:%d/%m %H:%M}</h5>'.format

problem_progress = IntProgress(
    value=0,
    min=0,
    max=n_submissions,
    step=1,
    description='Progress:',
    bar_style='success',
    orientation='horizontal',
)

## grader selection
grader = Dropdown(description='Grader', placeholder='Grader name', options=state.graders)
widgets_to_disable = (options_feedback, remarks)
for w in widgets_to_disable:
    traitlets.directional_link((state, 'grader_id'), (w, 'disabled'))

# 
option_names = Select(description='Feedback options', options=['ADD NEW'])
option_name = Text(placeholder='Option name')
option_score = IntText(description='Score')
save_option = Button(description='Update/add', button_style='success')
option_description = Textarea(placeholder='Full description', width='100%')
delete_option = Button(description='Delete', button_style='danger')

### Widget actions

## Utilities

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

def load_image(path):
    with open(path, 'rb') as f:
        return f.read()    

# render the solution widgets
def render_solution():
    with orm.db_session:
        if solution.graded_by is not None:
            graded_by = ' '.join((solution.graded_by.first_name, solution.graded_by.last_name))
        else:
            graded_by = ''
        num_graded = orm.count(s for s in db.Solution if s.problem.id == problem_id and s.graded_at is not None)

    # update the UI state
    answer.value = load_image(solution.image_path)

    # We also need to update one of the feedback options widgets
    option_names.options = ['ADD NEW'] + possible_feedback

    problem_progress.value = num_graded
    if solution.graded_at is None:
        graded_label.value = ''
    else: 
        graded_label.value = graded_label.template(graded_by, solution.graded_at or '')


## actual controllers - abusing decorator syntax FTW!

@prev_stud.on_click
def _(sender):
    state.previous_submission()

@next_stud.on_click
def _(sender):
    state.next_submission()

@next_ungraded_stud.on_click
def _(sender):
    state.next_ungraded()

@observe(jump_to_problem, names="value")
@save_then_render
def _(state):
    # 'problems' global var is list of *all* problems ordered by problem ID
    problem_id = state.problems.index(state['new']) + 1  # IDs in database start from 1
    
@observe(grader, names="value")
def _(state):
    global grader_id
    for w in widgets_to_disable:
        w.disabled = not bool(state['new'])
    try:
        grader_id = graders.index(state['new']) + 1  # IDs in database start from 1
    except ValueError:
        grader_id = None

@jump_to_stud.on_submit
def _(sender):
    global submission_id
    try:
        student_id = int(sender.value.strip()[:7])
    except ValueError:
        return
    state.jump_to_student(student_id)

## Controllers for the feedback options editing

@observe(option_names, names='value')
def _(state):
    value = state['new']
    if value == "ADD NEW":
        option_name.value = ''
        option_score.value = 0
        option_description.value = ''
        return

    with orm.db_session:
        fo = db.FeedbackOption.get(text=value, problem=db.Problem[problem_id])
        description = fo.description or ''
        score = fo.score or 0

    option_description.value = description
    option_score.value = score
    option_name.value = option_names.value


@save_option.on_click
def _(sender):
    value = option_names.value
    if value == "ADD NEW":
        value = option_name.value.strip()
        if not value:
            return
        with orm.db_session:
            if db.FeedbackOption.get(text=value, problem=db.Problem[problem_id]) is None:
                db.FeedbackOption(text=value, problem=db.Problem[problem_id])
            else:
                option_names.value = value

    # Set the details in the database.
    with orm.db_session:
        fo = db.FeedbackOption.get(text=value, problem=db.Problem[problem_id])
        fo.text = option_name.value.strip()
        fo.description = option_description.value.strip()
        fo.score = option_score.value

        possible_feedback = list(orm.select(f.text for f in db.FeedbackOption
                                            if f.problem.id == problem_id).order_by('f.id'))

        solution = db.Solution.get(submission=submission_id, problem=problem_id)
        selected_feedback = [f.text for f in solution.feedback]

    # Update UI state
    global loaded_feedback
    loaded_feedback = set(selected_feedback)
    
    # update the UI state
    options_feedback.options = possible_feedback
    options_feedback.value = selected_feedback

    # We also need to update one of the feedback options widgets
    option_names.options = ['ADD NEW'] + possible_feedback


@delete_option.on_click
def _(sender):
    value = option_names.value
    if value == "ADD NEW":
        return
    
    # Wipe the option from database and get new options
    with orm.db_session:
        db.FeedbackOption.get(text=value, problem=db.Problem[problem_id]).delete()

        orm.commit()
        possible_feedback = list(orm.select(f.text for f in db.FeedbackOption
                                            if f.problem.id == problem_id).order_by('f.id'))

        solution = db.Solution.get(submission=submission_id, problem=problem_id)
        selected_feedback = [f.text for f in solution.feedback]

    # Update UI state
    global loaded_feedback
    loaded_feedback = set(selected_feedback)
    
    # update the UI state
    options_feedback.options = possible_feedback
    options_feedback.value = selected_feedback

    # We also need to update one of the feedback options widgets
    option_names.options = ['ADD NEW'] + possible_feedback
    option_names.value = 'ADD NEW'

# Layout

In [ ]:
render_solution()

In [ ]:
HBox([Label(value="minitest2"), grader, jump_to_problem])

In [ ]:
# Main layout

grade = VBox([HBox([prev_stud, next_stud, next_ungraded_stud, jump_to_stud]),
              HBox([options_feedback, remarks]),
              HBox([problem_progress,  graded_label]), 
              answer])

edit_feedback = VBox([option_names, 
                      HBox([option_name, 
                            option_score,
                            save_option,
                            delete_option]),
                      
                      option_description])

editing = Tab(children=[grade, edit_feedback])
editing.set_title(0, 'Grade')
editing.set_title(1, 'Edit feedback options')
editing

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(jump_to_stud.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=students)))
display.display_javascript(prevent_unselect)